In [1]:
#read the csv file
import pandas as pd
can = pd.read_csv("https://raw.githubusercontent.com/medcab9/ds/master/data/Cannabis%20StrainsFeatures%20-%20cannabis.csv")
print(can.shape)
can.head()

(2351, 6)


,Strain,Type,Rating,Effects,Flavor,Description
0,100-Og,hybrid,4.0,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus",$100 OG is a 50/50 hybrid strain that packs a ...
1,98-White-Widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel",The ‘98 Aloha White Widow is an especially pot...
2,1024,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody",1024 is a sativa-dominant hybrid bred in Spain...
3,13-Dawgs,hybrid,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit",13 Dawgs is a hybrid of G13 and Chemdawg genet...
4,24K-Gold,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange","Also known as Kosher Tangie, 24k Gold is a 60%..."


In [2]:
can.describe(include='all')

,Strain,Type,Rating,Effects,Flavor,Description
count,2351,2351,2351.000000,2351,2350,2350
unique,2350,3,NaN,1704,1349,2343
top,B-Witched,hybrid,NaN,None,None,None
freq,2,1212,NaN,29,44,5
mean,NaN,NaN,4.382561,NaN,NaN,NaN
std,NaN,NaN,0.639090,NaN,NaN,NaN
min,NaN,NaN,0.000000,NaN,NaN,NaN
25%,NaN,NaN,4.200000,NaN,NaN,NaN
50%,NaN,NaN,4.500000,NaN,NaN,NaN
75%,NaN,NaN,4.700000,NaN,NaN,NaN


In [3]:
can.isnull().sum()

Strain         0
Type           0
Rating         0
Effects        0
Flavor         1
Description    1
dtype: int64

In [4]:
# Cleaning the data and changing nan values to unknowns and 0s
import numpy as np

can['Rating'] = can['Rating'].replace(np.nan, 0)
can['Effects'] = can['Effects'].replace(np.nan, 'Effects currently unknown')
can['Flavor'] = can['Flavor'].replace(np.nan, 'Flavor profile currently unavailable')
can['Description'] = can['Description'].replace(np.nan, 'Strain description currently unavailable')
can['Type'] = can['Type'].replace(np.nan, 'Type currently unknown')


In [5]:
can.isnull().sum()

Strain         0
Type           0
Rating         0
Effects        0
Flavor         0
Description    0
dtype: int64

In [6]:
# Unique Strain types

can.Strain.value_counts()

B-Witched            2
Sugar-Calyx          1
Jasmin               1
Zoom-Pie             1
Cello-Sweet-Og       1
                    ..
The-Grunk            1
Cascadia-Kush        1
Four-Star-General    1
Bordello             1
Candy-Cane           1
Name: Strain, Length: 2350, dtype: int64

In [7]:
can['Rating'].unique()

array([4. , 4.7, 4.4, 4.2, 4.6, 4.5, 4.3, 5. , 3.8, 4.1, 4.8, 3.4, 3.7,
       3.9, 0. , 3. , 4.9, 3.6, 2.8, 3.3, 3.5, 3.2, 1. , 3.1, 2.5])

In [12]:
# Create new series with information

can['total_text'] = can['Strain'] + can['Effects'] + can['Description']+can["Flavor"]

In [13]:
can.head()

,Strain,Type,Rating,Effects,Flavor,Description,total_text
0,100-Og,hybrid,4.0,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus",$100 OG is a 50/50 hybrid strain that packs a ...,"100-OgCreative,Energetic,Tingly,Euphoric,Relax..."
1,98-White-Widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel",The ‘98 Aloha White Widow is an especially pot...,"98-White-WidowRelaxed,Aroused,Creative,Happy,E..."
2,1024,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody",1024 is a sativa-dominant hybrid bred in Spain...,"1024Uplifted,Happy,Relaxed,Energetic,Creative1..."
3,13-Dawgs,hybrid,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit",13 Dawgs is a hybrid of G13 and Chemdawg genet...,"13-DawgsTingly,Creative,Hungry,Relaxed,Uplifte..."
4,24K-Gold,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange","Also known as Kosher Tangie, 24k Gold is a 60%...","24K-GoldHappy,Relaxed,Euphoric,Uplifted,Talkat..."


## 2) Use Spacy to tokenize the total text

In [9]:
import re
import string

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import spacy
from spacy.tokenizer import Tokenizer
from bs4 import BeautifulSoup

In [10]:
# Inialize spaCy model and tokenizer
nlp = spacy.load("en_core_web_lg")
tokenizer = Tokenizer(nlp.vocab)

In [14]:
tokens = []

# Return lower case tokens and ignore stop words/punctuation
for doc in tokenizer.pipe(can['total_text'], batch_size=500):
    
    doc_tokens = []
    
    for token in doc:
        # ignore stopwords, spaces and punctuation
        if (token.is_stop == False) & (token.is_punct == False) & (token.is_space == False):
            doc_tokens.append(token.text.lower())

    tokens.append(doc_tokens)

can['spaCy_tokens'] = tokens
can['spaCy_tokens'].head()

0    [100-ogcreative,energetic,tingly,euphoric,rela...
1    [98-white-widowrelaxed,aroused,creative,happy,...
2    [1024uplifted,happy,relaxed,energetic,creative...
3    [13-dawgstingly,creative,hungry,relaxed,uplift...
4    [24k-goldhappy,relaxed,euphoric,uplifted,talka...
Name: spaCy_tokens, dtype: object

In [15]:
print(can.shape)
can.head()

(2351, 8)


,Strain,Type,Rating,Effects,Flavor,Description,total_text,spaCy_tokens
0,100-Og,hybrid,4.0,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus",$100 OG is a 50/50 hybrid strain that packs a ...,"100-OgCreative,Energetic,Tingly,Euphoric,Relax...","[100-ogcreative,energetic,tingly,euphoric,rela..."
1,98-White-Widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel",The ‘98 Aloha White Widow is an especially pot...,"98-White-WidowRelaxed,Aroused,Creative,Happy,E...","[98-white-widowrelaxed,aroused,creative,happy,..."
2,1024,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody",1024 is a sativa-dominant hybrid bred in Spain...,"1024Uplifted,Happy,Relaxed,Energetic,Creative1...","[1024uplifted,happy,relaxed,energetic,creative..."
3,13-Dawgs,hybrid,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit",13 Dawgs is a hybrid of G13 and Chemdawg genet...,"13-DawgsTingly,Creative,Hungry,Relaxed,Uplifte...","[13-dawgstingly,creative,hungry,relaxed,uplift..."
4,24K-Gold,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange","Also known as Kosher Tangie, 24k Gold is a 60%...","24K-GoldHappy,Relaxed,Euphoric,Uplifted,Talkat...","[24k-goldhappy,relaxed,euphoric,uplifted,talka..."


## Part 2: Vector Representation

In [15]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer,TfidfTransformer

In [16]:
def gather_data(df_column):
    """ Produces List of Documents from a df column    
    returns list of strings 
    """
    
    #data = []
    
#     for row in df_column:
#         data.append(' '.join(str(row))
    data = [''.join(str(item)) for item in df_column]
        
    
    return data

In [17]:
data = gather_data(can['spaCy_tokens'])
data[0]

"['100-ogcreative,energetic,tingly,euphoric,relaxed$100', 'og', '50/50', 'hybrid', 'strain', 'packs', 'strong', 'punch.', 'supposedly', 'refers', 'strength', 'high', 'price', 'started', 'showing', 'hollywood.', 'plant,', '$100', 'og', 'tends', 'produce', 'large', 'dark', 'green', 'buds', 'stems.', 'users', 'report', 'strong', 'body', 'effect', 'indica', 'pain', 'relief', 'alert,', 'cerebral', 'feeling', 'thanks', 'sativa', 'side.earthy,sweet,citrus']"

In [18]:
from sklearn.feature_extraction.text import CountVectorizer

# create the transformer
vect = CountVectorizer()

# build vocab

vect.fit(data)

# transform text
dtm = vect.transform(data)
print(dtm.shape)

(2351, 12066)


In [19]:
# Tunning Parameters

# Instantiate vectorizer object
tfidf = TfidfVectorizer(stop_words='english',
                        ngram_range = (1,2),
                        max_df=.97,
                        min_df=3)

# Create a vocabulary and get word counts per document
dtm = tfidf.fit_transform(data)


# Get feature names to use as dataframe column headers
dtm = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names())

# View Feature Matrix as DataFrame
dtm.head()

,09,09 cookies,10,10 11,10 feet,10 week,10 weeks,100,100 sativa,11,...,yunnan,zealand,zest,zestful,zesty,zesty lemon,zion,zion botanicals,zombie,zombie og
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.452944,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Creating the model

In [20]:
from sklearn.neighbors import NearestNeighbors

# Fit on TF-IDF Vectors
nn  = NearestNeighbors(n_neighbors=5, algorithm='kd_tree')
nn.fit(dtm)

NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [30]:
# Test model

#fake_input = ['hybrid happy strawberry']
fake_input=[" I am looking for something which helps with my anxiety  and helps me go to sleep. I like it to have fruity flavour "]
fake_dense = tfidf.transform(fake_input)
_,fake_output = nn.kneighbors(fake_dense.todense())

fake_output

array([[ 411, 1889,  343, 2318,  153]], dtype=int64)

In [32]:
# Print output

response = can[['Strain', 'Effects', 'Flavor', 'Description']].iloc[fake_output[0]]
response


,Strain,Effects,Flavor,Description
411,Butterscotch,"Relaxed,Euphoric,Focused,Creative,Sleepy","Pine,Citrus,Nutty",Butterscotch is a treat of a strain that gets ...
1889,Skywalker,"Relaxed,Euphoric,Sleepy,Happy,Uplifted","Earthy,Sweet,Citrus",Skywalker is a well-rounded indica-dominant hy...
343,Blueberry-Muffins,"Relaxed,Euphoric,Uplifted,Happy,Creative","Blueberry,Earthy,Berry",Blueberry Muffins is the thoroughly baked indi...
2318,Wonder-Haze,"Happy,Euphoric,Creative,Energetic,Focused","Honey,Earthy,Pungent",Wonder Haze is a sativa-dominant hybrid strain...
153,B4,"Happy,Uplifted,Euphoric,Focused,Relaxed","Spicy/Herbal,Berry,Skunk","B4, bred by MTG Seeds, is a hybrid cannabis st..."


In [ ]:
# Example conversion to JSON

response.to_json(orient='records')

In [29]:
# # Create pickles of the model and the transformer for web deployment

# # Imports
# import pickle
# from sklearn.externals import joblib 

# # Save the model as a pickle file 
# joblib.dump(nn, 'knn_model.pkl') 

# # Save the transformer as a pickle file
# joblib.dump(tfidf, 'vector.pkl')

C:\Users\sseng\anaconda3\envs\U4-NLP\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


['vector.pkl']

In [22]:
# Write out fake 


fake_input= [" I am looking for something which helps with my anxiety  and helps me go to sleep. I like it to have fruity flavour "]

print(fake_input)

[' I am looking for something which helps with my anxiety  and helps me go to sleep. I like it to have fruity flavour ']


In [23]:
# new = tfidf.transform(fake_input)
# new

<1x9694 sparse matrix of type '<class 'numpy.float64'>'
	with 6 stored elements in Compressed Sparse Row format>

In [43]:
# nn.kneighbors(new.todense())

(array([[1.30223187, 1.30695919, 1.31023683, 1.32285729, 1.32408118]]),
 array([[ 411, 1889,  343,  153, 2318]], dtype=int64))

In [45]:
# # Print two results

# print("Recommendation 1---",can['total_text'][411])

# print("Recommendation 2---",can['total_text'][1889])

Recommendation 1--- ButterscotchRelaxed,Euphoric,Focused,Creative,SleepyButterscotch is a treat of a strain that gets its name from its distinctive candy aroma. This strong indica helps patients to gain some mental peace and unwind from a stressful day. Effects tend to be sedative and sleep-inducing, so Butterscotch is a good choice for treating insomnia.Pine,Citrus,Nutty
Recommendation 2--- SkywalkerRelaxed,Euphoric,Sleepy,Happy,UpliftedSkywalker is a well-rounded indica-dominant hybrid that helps patients knock out pain and relax into sleep after a long day of battling the Dark Side. The sativa side of this strain allows for a nice head high as well, but it isn’t overly hazy or unfocused. Skywalker was created by Amsterdam’s Dutch Passion Seeds to combine the sweet fruity aroma of Blueberry with the growing strengths of the hardy, high-yielding Mazar. Great for indoor growing, Skywalker plants will stay shorter and bushy, flowering in 8 to 9 weeks. If you’re looking for the Californi

In [24]:
# response = can[['Strain','Type', "Rating",'Effects', 'Flavor', 'Description']].iloc[[411]]
# response

,Strain,Type,Rating,Effects,Flavor,Description
411,Butterscotch,indica,4.3,"Relaxed,Euphoric,Focused,Creative,Sleepy","Pine,Citrus,Nutty",Butterscotch is a treat of a strain that gets ...


In [25]:
# response.to_json(orient='records')

'[{"Strain":"Butterscotch","Type":"indica","Rating":4.3,"Effects":"Relaxed,Euphoric,Focused,Creative,Sleepy","Flavor":"Pine,Citrus,Nutty","Description":"Butterscotch is a treat of a strain that gets its name from its distinctive candy aroma. This strong indica helps patients to gain some mental peace and unwind from a stressful day. Effects tend to be sedative and sleep-inducing, so Butterscotch is a good choice for treating insomnia."}]'

In [26]:
# import pickle

# with open('knn_model.pkl', 'wb') as f:
#     pickle.dump(nn, f)

In [ ]:
# # Instantiate vectorizer object
# tfidf = TfidfVectorizer(stop_words='english',
#                         ngram_range = (1,2),
#                         max_df=.97,
#                         min_df=3)

# # Create a vocabulary and get word counts per document
# dtm = tfidf.fit_transform(input)


# # Get feature names to use as dataframe column headers
# dtm = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names())

# # View Feature Matrix as DataFrame
# dtm.head()

In [ ]:
# pickle.dump(tfidf, open("vectorizer.pickle", "wb"))

In [ ]:
## have to run the code block
## have to tokenize the input before vectorizing it